## Note: mode1-taxi, mode2-FHV, mode3-shared FHV, mode4-PT, mode5-walking

In [32]:
import numpy as np
import pandas as pd
import warnings
random_seed = 1
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from tqdm import tqdm
np.random.seed(2019)
%matplotlib inline

In [3]:
location = pd.read_csv('../cleaned_data/lat_long_for_API.csv')
location['ODpair'] = location['LocationID_pickup'].astype(str) + '-' + location['LocationID_dropoff'].astype(str)
location_list = location['ODpair'].unique()
location.head()

,Unnamed: 0,LocationID_pickup,lon_pickup,lat_pickup,LocationID_dropoff,lon_dropoff,lat_dropoff,ODpair
0,1,1,-74.176778,40.689515,2,-73.826141,40.625724,1-2
1,2,1,-74.176778,40.689515,3,-73.849479,40.865871,1-3
2,3,1,-74.176778,40.689515,4,-73.977024,40.724151,1-4
3,4,1,-74.176778,40.689515,5,-74.189938,40.550339,1-5
4,5,1,-74.176778,40.689515,6,-74.067786,40.599053,1-6


## Data Preparation

In [4]:
acs = pd.read_csv('final_acs_transportation_choice.csv')
acs.head()

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
0,3.0,0.228957,35.391246,14.547350,7706.507979,949.324468
1,4.0,46.244797,152.049702,91.310873,7487.249289,2840.145340
2,5.0,0.349401,56.498123,9.104389,7989.174863,173.873224
3,6.0,0.263186,18.439186,4.534672,4767.467108,478.295847
4,7.0,29.894066,167.041570,44.019826,33307.536619,3139.507920


In [5]:
mode_data = pd.read_csv('final_allMode_with_wage_cleaned_update.csv', index_col=0)
mode_data = mode_data.dropna()
print(mode_data.shape)
mode_data.head()

(242834, 17)


,12500,125000,17500,22500,225000,2500,30000,42500,62500,7500,87500,DOlocationID,PUlocationID,duration,mode,nest,price
ODpair,,,,,,,,,,,,,,,,,
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.93859,0.360216,0.586936,4,3,39.695000,2,1,64.000000
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.93859,0.360216,0.586936,4,3,45.216667,3,1,61.500000
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.93859,0.360216,0.586936,4,3,83.000000,4,2,5.500000
3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.93859,0.360216,0.586936,4,3,225.933333,5,3,0.000000
3-7,3.890709,4.444350,1.693267,2.488532,3.084029,1.504915,3.552056,4.027693,5.63154,2.161294,3.521615,7,3,47.880952,1,1,43.157143


## Modeling

### Nested Logit Model

In [6]:
def utility(mode, wage, Lambda, dataset):
    '''
    Get the utility for mode j under specific OD pair
    Lambda: parameter that trade-off different transportation mode
    '''
    subset = dataset[dataset['mode'] == mode]
    vj = Lambda * (float(int(wage)/124800) * float(subset['duration']) + float(subset['price'])) #525600: convert wage scale 'year' to 'minitues'
    return -vj #assign negative utility?


def InclusiveValue(Nk, Tk, wage, dictVj, Lambda, dataset):
    '''
    Get the inclusive value for nest K
    Nk:nest k
    T: the dict that contains Tao(dissmilarity parameter) for each Nest. for example, Tk meeas Tao for nest K
    '''
    subsetNk = dataset[dataset['nest'] == Nk]
    modes = list(subsetNk['mode'].unique()) #what modes contained in this nest k
    sumIV = 0
    for j in modes:   
        vj = utility(j, wage, Lambda, subsetNk)
        dictVj[j] = vj
        sumIV += np.exp((1/Tk)*vj)  
    IVk = np.log(sumIV)
    return dictVj, IVk
    

def denoSum(T, nestList, wage, Lambda, dataset):
    '''
    Calculate the denomenator for P(y=Nk)
    T: the dict that contains Tk for each Nest, in our case T={1:T1, 2:T2, 3:T3}; T1, T2, T3 defined by us 
    TotK: the total number of nest this model has, in our case TotK = 3
    '''
    deno = 0
    dictIVk = {}
    dictVj = {}
    for Nk in nestList: #k is the k Nest, in our case k=1,2,3
        Tk = T[Nk] #get the tao for nest k 
        dictVj, IVk = InclusiveValue(Nk, Tk, wage, dictVj, Lambda, dataset)
        denok = np.exp(Tk*IVk)
        deno += denok
        dictIVk[Nk] = IVk
    return dictVj, dictIVk, deno


def probability (j, Nk, T, dictVj, dictIVk, deno):  
    '''
    Calculate the probability for the specific mode j and Nest Nk
    '''
    Tk = T[Nk]
    IVk = dictIVk[Nk]
    vj = dictVj[j]
    pjk = np.exp((1/Tk)*vj)/np.exp(IVk) #pjk: P(y=j, y belong to Nk)
    pk = np.exp(Tk*IVk)/deno #pk: P(y belong to Nk)
    pj = pjk*pk #pj: P(y=j)
    return pj

### Apply to our case

In [18]:
def apply_model(T, wageList, Lambda, dataAll):
    '''
    apply to our case
    '''
    import timeit
    start = timeit.default_timer()

    ODpair_list = list(dataAll.index.unique())
    p = []
    loss_function_deno = []
    for j in tqdm(ODpair_list,position=0): #indentify OD pair  
        dataOD = dataAll[dataAll.index==j] 
        modeList = list(dataOD['mode'])
        nestList = list(dataOD['nest'])
        
        pop_OD = [] #store the results under each OD pair
        loss_function_deno_OD = []
        for wage in wageList:            
            dictVj, dictIVk, deno = denoSum(T, set(nestList), wage, Lambda, dataOD)
            pop_mode = [] #store the population results under each OD pair and each wage
            loss_function_deno_mode = [] #store the deno of loss function under each OD pair and each wage
            for i in range(1,6):
                if i in modeList: #not all modes appear in every OD pair
                    prob = probability(i, nestList[modeList.index(i)], T, dictVj, dictIVk, deno) #probability under OD pari and mode i
                    if np.isnan(prob) == True: #if predicted probability is nan, replace it as 0, means no people choose
                        prob = 0
                    
                    pop = dataOD[wage].mean() * prob
                    loss_function_denoi = dataOD[wage].mean() * (prob - prob**2)               
                    pop_mode.append(pop)
                    loss_function_deno_mode.append(loss_function_denoi)
                else: 
                    pop_mode.append(0) #the probability of mode which not in the modeList is also 0
                    loss_function_deno_mode.append(0)
                    #pop_mode.append(1)
            pop_OD.append(pop_mode) #the shape of pop_OD is len(wageList) * 5 
            loss_function_deno_OD.append(loss_function_deno_mode)
        pop_OD_sum = [sum(x) for x in zip(*pop_OD)] #sum the population of each wage 
        loss_function_deno_sum = [sum(x) for x in zip(*loss_function_deno_OD)]
        p.append(pop_OD_sum)
        loss_function_deno.append(loss_function_deno_sum)

    df = pd.DataFrame(p, columns=['P(mode1)', 'P(mode2)', 'P(mode3)', 'P(mode4)', 'P(mode5)']
                      , index=ODpair_list)
    df_LF_deno = pd.DataFrame(loss_function_deno, columns=['mode1', 'mode2', 'mode3', 'mode4', 'mode5']
                               , index=ODpair_list)
    
    stop = timeit.default_timer()
    timeslot = stop - start
    return df, df_LF_deno, timeslot

def compare_with_ground_truth(predictdf, loss_function_deno, truedf):
    '''
    compare our predicted transportation choice with ground truth
    
    The header of the datafrme after merge (named 'data_compare') should be like:
    taxi_zone | P(mode1)_x | P(mode2)_x | P(mode3)_x | P(mode4)_x | P(mode5)_x | P(mode1)_y | P(mode2)_y | P(mode3)_y | P(mode4)_y | P(mode5)_y
    '''
    import numpy as np
    # makesure predictdf and truedf have the same formats
    for col in predictdf.columns:
        predictdf[col] = predictdf[col].astype(float) 
    predictdf = predictdf.fillna(0)
    predictdf = predictdf.replace([np.inf, -np.inf], np.nan)
    predictdf = predictdf.dropna()
    predictdf['taxi_zone'] = predictdf.index.map(lambda x: x.split('-')[0]) #get origin taxi zone from each OD pair
    predictdf = predictdf.groupby('taxi_zone').sum().reset_index() #group the popuation by taxi zone
    predictdf['taxi_zone'] = predictdf['taxi_zone'].astype(int)
    
    #do same thing as predictdf to loss_function_deno dataframe
    for col in loss_function_deno.columns:
        loss_function_deno[col] = loss_function_deno[col].astype(float) 
    loss_function_deno = loss_function_deno.fillna(0)
    loss_function_deno = loss_function_deno.replace([np.inf, -np.inf], np.nan)
    loss_function_deno = loss_function_deno.dropna()
    loss_function_deno['taxi_zone'] = loss_function_deno.index.map(lambda x: x.split('-')[0]) #get origin taxi zone from each OD pair
    loss_function_deno = loss_function_deno.groupby('taxi_zone').sum().reset_index() #group the popuation by taxi zone
    loss_function_deno['taxi_zone'] = loss_function_deno['taxi_zone'].astype(int)   
    
    truedf['taxi_zone'] = truedf['taxi_zone'].astype(int)
        
    data_compare = pd.merge(predictdf, truedf, left_on='taxi_zone', right_on = 'taxi_zone', how = 'left')
    data_compare = data_compare.dropna() 
    
    loss = 0
    rloss = 0
    for i in range(1,6):
        #define the loss function
        lossi = sum((data_compare[data_compare.columns[i]] - data_compare[data_compare.columns[i+5]])**2/loss_function_deno[loss_function_deno.columns[i]])
        rlossi = np.sqrt(lossi)
    loss += lossi
    rloss += rlossi
    return data_compare, loss_function_deno, loss, rloss

In [19]:
#choose ODpair 3-1 to test the algrithm whether bug-free:
T1 = 3
T2 = 1
T3 = 1
T = {1:T1, 2:T2, 3:T3} #Tao for each nest
Lambda = 1 
wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']
# testdf = mode_data[mode_data.index.isin(list(mode_data.index[:20]))] #test OD pair 3-1,3-2,3-3,3-4
testdf = mode_data[mode_data['PUlocationID']<=4]
# testdf = mode_data[mode_data.index == '3-2']

predict_choice_test, loss_function_denodf_test, timeslot_test = apply_model(T, wagelist, Lambda, testdf)
combine_test, loss_deno_test, loss_test, rloss_test = compare_with_ground_truth(predict_choice_test, loss_function_denodf_test, acs)
print('The time used to run the code:', timeslot_test)
print('The weighted cumulative square error of this model is:', loss_test)
print('The root of weighted cumulative square error of this model is:', rloss_test)
print()
print('The predict transportation choice is:')
predict_choice_test.head()

100%|██████████| 471/471 [00:20<00:00, 19.34it/s]

The time used to run the code: 20.74090717199988
The weighted cumulative square error of this model is: 419.08216068430613
The root of weighted cumulative square error of this model is: 20.471496298128923

The predict transportation choice is:


,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
3-4,0.000000,4.744689e-01,0.039536,5.236155,0.249839
3-7,0.003762,3.291113e+00,0.174238,31.139544,1.391343
3-9,0.000000,6.040043e-01,0.025470,1.305387,0.065139
3-10,0.000000,6.362928e-01,0.000003,2.241670,0.122034
3-11,0.000000,8.805902e-33,0.000000,3.868835,0.131165


In [9]:
combine_test

,taxi_zone,P(mode1)_x,P(mode2)_x,P(mode3)_x,P(mode4)_x,P(mode5)_x,P(mode1)_y,P(mode2)_y,P(mode3)_y,P(mode4)_y,P(mode5)_y
0,3,709.266929,867.421554,448.755073,5931.400161,749.156283,0.228957,35.391246,14.547350,7706.507979,949.324468
1,4,3193.065730,534.047886,322.544560,4014.329852,2553.011971,46.244797,152.049702,91.310873,7487.249289,2840.145340


In [10]:
loss_deno_test

,taxi_zone,mode1,mode2,mode3,mode4,mode5
0,3,277.148736,377.689921,297.885906,272.997076,183.109027
1,4,781.809272,397.097915,277.811048,681.119768,411.681312


### Tune the parameters

In [33]:
rloss_best = np.inf
rloss_all = []

for Tau in np.logspace(-2,2,5):
    for lamb in np.logspace(-2,2,5):
        T1 = Tau #should set grid search for T1 
        T2 = 1
        T3 = 1
        T = {1:T1, 2:T2, 3:T3} #Tao for each nest
        Lambda = lamb #should set grid search for Lambda 
        nestList = [1, 2, 3]
        wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']
        #randomly pick 10 taxizone to tune the parameters
        test_zone = np.random.randint(min(mode_data['PUlocationID']), max((mode_data['PUlocationID'])), 10)
        testdf = mode_data[mode_data['PUlocationID'].isin(test_zone)]

        predict_choice_test,loss_function_denodf_test, timeslot_test = apply_model(T, wagelist, Lambda, testdf)
        results,loss_deno_test, loss_test, rloss_test = compare_with_ground_truth(predict_choice_test,loss_function_denodf_test, acs)
        rloss_all.append(rloss_test)
        if rloss_test < rloss_best:
            rloss_best = rloss_test
            T1_best = T1
            Lambda_best = Lambda
            results_best = results.copy()
    
print(rloss_best,Lambda_best,T1_best)
        

100%|██████████| 2014/2014 [01:23<00:00, 24.18it/s]


91.92116992662218 0.01 0.01


In [30]:
rloss_all

In [34]:
rloss_best_2 = np.inf
rloss_all_2 = []

for Tau in np.logspace(-2,2,10):
    for lamb in np.logspace(-2,2,10):
        T1 = Tau #should set grid search for T1 
        T2 = 1
        T3 = 1
        T = {1:T1, 2:T2, 3:T3} #Tao for each nest
        Lambda = lamb #should set grid search for Lambda 
        nestList = [1, 2, 3]
        wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']
        #randomly pick 10 taxizone to tune the parameters
        test_zone = np.random.randint(min(mode_data['PUlocationID']), max((mode_data['PUlocationID'])), 10)
        testdf = mode_data[mode_data['PUlocationID'].isin(test_zone)]

        predict_choice_test,loss_function_denodf_test, timeslot_test = apply_model(T, wagelist, Lambda, testdf)
        results,loss_deno_test, loss_test, rloss_test = compare_with_ground_truth(predict_choice_test,loss_function_denodf_test, acs)
        rloss_all_2.append(rloss_test)
        if rloss_test < rloss_best_2:
            rloss_best_2 = rloss_test
            T1_best_2 = T1
            Lambda_best_2 = Lambda
            results_best_2 = results.copy()
    
print(rloss_best_2,Lambda_best_2,T1_best_2)
        

 33%|███▎      | 766/2333 [00:36<01:14, 21.14it/s]

KeyboardInterrupt: 

 33%|███▎      | 766/2333 [00:55<01:14, 21.14it/s]

In [35]:
print(rloss_best_2,Lambda_best_2,T1_best_2)

66.85287596593618 0.21544346900318834 0.01


In [11]:
#randomly pick 10 taxizone to tune the parameters:
rloss_best = 1e10
best_T1 = 3
best_Lambda = 1

for Ti in [0.1, 0.3, 0.5]:
    for lambdai in [0.1, 0.3]:
        T1 = Ti
        T2 = 1
        T3 = 1
        T = {1:T1, 2:T2, 3:T3} #Tao for each nest
        Lambda = lambdai 
        wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']
        test_zone = np.random.randint(min(mode_data['PUlocationID']), max((mode_data['PUlocationID'])), 10)
        testdf = mode_data[mode_data['PUlocationID'].isin(test_zone)]
        
    
        predict_choice_test, loss_function_denodf_test, timeslot_test = apply_model(T, wagelist, lambdai, testdf)
        combine_test, loss_deno_test, loss_test, rloss_test = compare_with_ground_truth(predict_choice_test, loss_function_denodf_test, acs)
        if rloss_test < rloss_best:
            rloss_best = rloss_test
            best_T1 = Ti
            best_Lambda = lambdai
print(rloss_best, best_T1, best_Lambda)

192.5010912793663 0.5 0.3


## Apply to Scenario 1

In [36]:
#Run the model for the whole dataset
T1 = T1_best_2 #should set grid search for T1 
T2 = 1
T3 = 1
T = {1:T1, 2:T2, 3:T3} #Tao for each nest
Lambda = Lambda_best_2 #should set grid search for Lambda 
wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']

predict_choice, loss_function_denodf, timeslot = apply_model(T, wagelist, Lambda, mode_data)
combine, loss_deno, loss, rloss = compare_with_ground_truth(predict_choice, loss_function_denodf, acs)

100%|██████████| 55798/55798 [56:39<00:00, 16.41it/s]


NameError: name 'predict_transportation_choice' is not defined

In [37]:
combine, loss_deno, loss, rloss = compare_with_ground_truth(predict_choice, loss_function_denodf, acs)

In [38]:
print('The time used to run the code:', timeslot)
print('The rmse of this model is:', rloss)
print()
print('The predict transportation choice is:')
predict_choice.head()

The time used to run the code: 2981.28500231
The rmse of this model is: nan

The predict transportation choice is:


,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
3-4,0.0,0.0,0.0,5.614980,0.385020
3-7,0.0,0.0,0.0,34.045745,1.954255
3-9,0.0,0.0,0.0,1.944573,0.055427
3-10,0.0,0.0,0.0,2.898314,0.101686
3-11,0.0,0.0,0.0,3.846565,0.153435


In [40]:
predict_choice.to_csv('results_scenario17_7_1.csv')
combine.to_csv('results_scenario1_combined_17_7.csv')
loss_function_denodf.to_csv('loss_function_denodf.csv')
loss_deno.to_csv('loss_deno.csv')

In [20]:
print('The combined dataset is:')
combine.head()

The combined dataset is:


,taxi_zone,P(mode1)_x,P(mode2)_x,P(mode3)_x,P(mode4)_x,P(mode5)_x,P(mode1)_y,P(mode2)_y,P(mode3)_y,P(mode4)_y,P(mode5)_y
0,10,713.628413,2001.066133,503.898413,6277.929802,1363.477240,11.652287,149.882786,63.664064,9899.616882,736.183981
1,100,152.711374,3.265568,11.869164,527.050722,7.103172,48.244337,13.880226,2.623010,293.067961,344.184466
2,101,803.916232,1063.069978,486.140935,3004.639809,779.233046,0.785213,46.909507,7.434348,5191.320614,893.550318
3,102,2697.431255,1570.110234,254.704151,6866.190158,2222.564202,0.346400,66.242517,22.211104,12097.874199,1428.325779
4,106,668.268195,89.952342,30.016407,2412.792514,60.970542,0.763520,15.625191,2.837223,2986.428291,256.345776


In [22]:
#Save df to csv
predict_transportation_choice.to_csv('results_scenario1_7_3.csv')
combine.to_csv('results_scenario1_combined_7_3.csv')

### Apply to scenario 2 and 3
Only use effeccted taxi zone for Scenario2 (+2.75) and scenario 3 (+10):

**For Scenario 2**
Taxi-zones below 96th street: 140,141, 237, 236, 263, 262, 43, 238, 239, 143,142, 12, 88, 261, 13, 87, 209, 231, 45, 232, 148, 144, 211, 125, 158, 249, 114, 113, 79, 4, 224, 107, 234, 90, 68, 246, 186, 164, 100, 170, 137, 233, 162, 161, 230, 48, 50, 163, 229.

**For Scenario 3**
Taxi-zones under 60th street: 12, 88, 261, 13, 87, 209, 231, 45, 232, 148, 144, 211, 125, 158, 249, 114, 113, 79, 4, 224, 107, 234, 90, 68, 246, 186, 164, 100, 170, 137, 233, 162, 161, 230, 48, 50, 163, 229.